# 1. Cài đặt môi trường

In [1]:
# Import các thư viện cần thiết
import requests
import numpy as np
import pandas as pd
import json

# 2. Phân tích Web API cung cấp dữ liệu

- Trong phần này, công việc của tôi là thu thập dữ liệu sử dụng Web API từ (https://ghoapi.azureedge.net/api/MORT_100). Đây là dữ liệu của Tổ chức Y tế Thế giới trong đó bao gồm dữ liệu về số lượng ca tử vong ở trẻ em dưới 5 tuổi theo từng nguyên nhân của tất cả các quốc gia trên thế giới từ năm 2000 đến năm 2021.
- Indicator MORT_100 (Số lượng ca tử vong ở trẻ em dưới 5 tuổi theo từng nguyên nhân) có các Dimension sau:
    + `SpatialDimType`: Thuộc tính chiều không gian của dữ liệu, giá trị có thể là COUNTRY hoặc REGION
    + `SpatialDim`: Giá trị cụ thể của `SpatialDimType`, có thể là `VNM`, `USA`, `KOR`,... (mã của các quốc gia)
    + `TimeDimType`: Thuộc tính chiều thời gian của dữ liệu, giá trị là `YEAR`, chia dữ liệu theo năm
    + `TimeDim`: Giá trị cụ thể của `TimeDimType`, giá trị từ `2000 - 2021`
    + `Dim1Type`: Thuộc tính nhóm tuổi của dữ liệu, giá trị là `AGEGROUP`
    + `Dim1`: Giá trị cụ thể của `Dim1Type`, nhóm tuổi của dữ liệu được xác định theo ngày hoặc tháng hoặc năm
    + `Dim2Type`: Thuộc tính nguyên nhân tử vong của dữ liệu, giá trị là `CHILDCAUSE`
    + `Dim2`: Giá trị cụ thể của `Dim2Type`, là các mã của các nguyên nhân tử vong như `CH10`, `CH11`,...
    + `NumericValue`: Giá trị của số lượng ca tử vong ở trẻ em dưới 5 tuổi thoả các thuộc tính ở trên

- Dữ liệu ta muốn thu thập là dữ liệu được lọc theo
    + `SpatialDimType` = `COUNTRY`: Dữ liệu được chia theo từng quốc gia  
    + `Dim1` = `YEARS0-4`: Dữ liệu được chia theo nhóm tuổi xác định theo năm, từ năm thứ 0 đến năm thứ 4 (dưới 5 tuổi)


In [2]:
# URL ban đầu
BASE_URL = 'https://ghoapi.azureedge.net/api/MORT_100'

# Lọc theo Dim1
BASE_URL += '?$filter=Dim1' + '%20eq%20' + '%27YEARS0-4%27'

# Lọc theo SpatialDimType
BASE_URL += '%20and%20' + 'SpatialDimType' + '%20eq%20' + '%27COUNTRY%27'

# Tạo các dict và list lưu trữ dữ liệu về quốc gia, năm, loại nguyên nhân tử vong
COUNTRIES = {"Country_ID": [], "Country_Name": []}
YEARS = (list(range(2000, 2022)))
CAUSES = {"Causes_ID": [], "Causes_Name": []}

# Dict lưu trữ số lượng ca tử vong dựa trên các thuộc tính quốc gia, năm và nguyên nhân
NUMBER_OF_DEATHS = {}

print('BASE_URL:', BASE_URL)


BASE_URL: https://ghoapi.azureedge.net/api/MORT_100?$filter=Dim1%20eq%20%27YEARS0-4%27%20and%20SpatialDimType%20eq%20%27COUNTRY%27


# 3. Sử dụng Web API thu thập dữ liệu theo từng bước

- Ta có `BASE_URL` ở trên để gọi API lấy toàn bộ dữ liệu về số lượng ca tử vong của trẻ em theo các nguyên nhân, ở từng quốc gia với nhóm tuổi từ năm thứ 0 đến năm thứ 4.
- Dữ liệu DataFrame ta muốn có được bao gồm các cột nguyên nhân tử vong, 1 cột quốc gia và 1 cột năm; các hàng dữ liệu thể hiện số lượng ca tử vong của trẻ em bởi từng nguyên nhân ở quốc gia cụ thể và năm khảo sát cụ thể.
- Như vậy DataFrame sẽ có:
    + Số cột = số cột nguyên nhân tử vong + 2 `(quốc gia và năm)`
    + Số hàng = số quốc gia * số năm = số quốc gia * 22 `(2000 - 2021)`
    + Dữ liệu được thống kê `theo từng năm` từ 2000 đến 2021, ở mỗi năm thống kê theo `từng quốc gia`
- Các bước để thu thập dữ liệu mong muốn
    + Thu thập dữ liệu về các nguyên nhân tử vong ở trẻ em bao gồm mã nguyên nhân và tên nguyên nhân
    + Thu thập dữ liệu về các quốc gia được khảo sát trong Indicator MORT_100 bao gồm mã quốc gia và tên quốc gia
    + Với từng năm từ `2000 - 2021`, từng mã nguyên nhân, thu thập dữ liệu số ca tử vong của tất cả quốc gia, sắp xếp theo bảng chữ cái dựa trên mã quốc gia
    + Với các nguyên nhân không có dữ liệu, điền vào các giá trị None ứng với số lượng các quốc gia
    + Sau khi có được dữ liệu cho tất cả các cột nguyên nhân tử vong của các quốc gia, điền tên các quốc gia đã được thu thập trước vào cột quốc gia
    + Điền giá trị của năm hiện đang thu thập dữ liệu vào cột năm cho từng quốc gia


In [3]:

# Hàm thu thập dữ liệu về các nguyên nhân tử vong ở trẻ em dưới 5 tuổi
def collect_Causes():

    # Gọi API lấy các giá trị của thuộc tính CHILDCAUSE trong dữ liệu của WHO về Dimension
    # CHILDCAUSE: Các nguyên nhân tử vong ở trẻ em
    url = "https://ghoapi.azureedge.net/api/DIMENSION/CHILDCAUSE/DimensionValues"

    # Lấy dữ liệu từ API, tạo thành DataFrame với 2 cột là mã nguyên nhân (Code) và tên nguyên nhân (Title)
    response = requests.get(url)
    causes_data = response.json()['value']
    causes_df = pd.DataFrame(causes_data).loc[:, ["Code", "Title"]]

    # Lưu DataFrame vừa tạo vào dict CAUSES có các key ứng với từng cột của DataFrame
    if len(CAUSES["Causes_ID"]) == 0:
        for i in range(len(causes_df)):
            CAUSES["Causes_ID"].append(causes_df.iloc[i]['Code'])
            CAUSES["Causes_Name"].append(causes_df.iloc[i]['Title'])
            NUMBER_OF_DEATHS[causes_df.iloc[i]['Title']] = []

    # Trả về DataFrame tạo bởi dict CAUSES
    return pd.DataFrame(CAUSES)


In [4]:
# Lưu DataFrame nguyên nhân tử vong vào file csv, có 2 cột là ID và Name, với tổng 17 nguyên nhân
causes_df = collect_Causes()
pd.set_option('max_colwidth', 100)
causes_df.to_csv("../../../Data/causes_of_death.csv", index=False)
causes_df

,Causes_ID,Causes_Name
0,CH10,Prematurity
1,CH11,Birth asphyxia and birth trauma
2,CH12,Sepsis and other infectious conditions of the newborn
3,CH13,Other Group 1 (postneonatal only)
4,CH15,Congenital anomalies
5,CH16,Other noncommunicable diseases (postneonatal only)
6,CH17,Injuries
7,CH18,Tuberculosis
8,CH19,Other Group 1 and Other noncommunicable (neonatal and under-5 only)
9,CH2,HIV/AIDS


In [5]:
# Hàm thu thập dữ liệu về các quốc gia được thống kê trong Indicator MORT_100
def collect_Countries():

    # Gọi API của BASE_URL lấy dữ liệu được lọc theo từng nguyên nhân tử vong (Dim2) và năm khảo sát (TimeDim)
    url = BASE_URL + "%20and%20Dim2%20eq%20%27" + CAUSES["Causes_ID"][0]
    url += "%27%20and%20TimeDim%20eq%20" + str(YEARS[0])

    # Dữ liệu thu được là số lượng ca tử vong của 1 nguyên nhân và 1 năm khảo sát cụ thể
    # Từ đó lấy ra danh sách các quốc gia được khảo sát trong Indicator Mort_100, từ Dimension SpatialDim
    response = requests.get(url)
    countries_data = response.json()['value']

    # Dữ liệu chỉ gồm 1 cột thể hiện mã của các quốc gia
    countries_df = pd.DataFrame(countries_data).loc[:, ["SpatialDim"]]

    # Sắp xếp theo thứ tự bảng chữ cái
    countries_df['SpatialDim'] = sorted(countries_df['SpatialDim'])

    # Lưu DataFrame vừa tạo vào dict COUNTRIES có các key ứng với từng cột của DataFrame
    # key không có value thì gán các giá trị bằng None
    if len(COUNTRIES["Country_ID"]) == 0:
        for i in range(len(countries_df)):
            COUNTRIES["Country_ID"].append(countries_df.iloc[i]['SpatialDim'])
            COUNTRIES["Country_Name"].append(None)

    # Gọi API lấy các giá trị của thuộc tính COUNTRY trong dữ liệu của WHO về Dimension
    # COUNTRY: Danh sách các quốc gia
    url = "https://ghoapi.azureedge.net/api/DIMENSION/COUNTRY/DimensionValues"

    # Lấy dữ liệu từ API, tạo thành DataFrame với 2 cột là mã quốc gia (Code) và tên tên quốc gia (Title)
    response = requests.get(url)
    countries_data = response.json()['value']
    countries_df = pd.DataFrame(countries_data).loc[:, ["Code", "Title"]]

    # Lọc DataFrame theo các quốc gia được khảo sát trong Indicator MORT_100 (các quốc gia lưu trong dict COUNTRIES)
    # Các quốc gia còn lại thì bỏ qua
    countries_df = countries_df[countries_df['Code'].isin(COUNTRIES["Country_ID"])].reset_index(drop=True)

    # Lưu các giá trị của tên quốc gia vào dict COUNTRIES
    COUNTRIES["Country_Name"] = countries_df["Title"]

    # Trả về DataFrame tạo bởi dict COUNTRIES
    return pd.DataFrame(COUNTRIES)


In [6]:
# Lưu DataFrame các quốc gia vào file csv, có 2 cột là ID và Name, với tổng 194 quốc gia
countries_df = collect_Countries()
countries_df.to_csv("../../../Data/countries.csv", index=False)
countries_df

,Country_ID,Country_Name
0,AFG,Afghanistan
1,AGO,Angola
2,ALB,Albania
3,AND,Andorra
4,ARE,United Arab Emirates
...,...,...
189,WSM,Samoa
190,YEM,Yemen
191,ZAF,South Africa
192,ZMB,Zambia


In [7]:
# Hàm thu thập dữ liệu số lượng ca tử vong theo nguyên nhân tử vong, quốc gia và năm khảo sát
def collect_data():
    # Tạo 2 mảng rỗng để lưu các giá trị của 2 cột thuộc tính quốc gia và năm
    NUMBER_OF_DEATHS["CountryName"] = []
    NUMBER_OF_DEATHS["Year"] = []

    # Điền các giá trị vào dict NUMBER_OF_DEATHS theo từng năm
    for year in YEARS:
        for cause_id, cause_name in zip(CAUSES["Causes_ID"], CAUSES["Causes_Name"]):
            # Với từng nguyên nhân tử vong ở 1 năm cụ thể, điền dữ liệu vào các cột nguyên nhân và cột năm
            # Mỗi nguyên nhân tử vong là 1 cột

            # Tạo url để gọi API lấy dữ liệu số lượng ca tử vong lọc theo nguyên nhân và năm cụ thể
            url = BASE_URL + "%20and%20Dim2%20eq%20%27" + cause_id + "%27%20and%20TimeDim%20eq%20" + str(year)
            df = pd.DataFrame(requests.get(url).json()['value'])

            # Kiểm tra dữ liệu tồn tại
            if "SpatialDim" in df.columns and "NumericValue" in df.columns:
                # Điền các giá trị số ca tử vong của nguyên nhân cụ thể theo quốc gia sắp xếp theo bảng chữ cái
                # để đối ứng với dict COUNTRIES đã được lưu từ trước
                df = pd.DataFrame(requests.get(url).json()['value']).loc[:, ["SpatialDim", "NumericValue"]]
                NUMBER_OF_DEATHS[cause_name].extend(list(df.sort_values('SpatialDim').reset_index(drop=True)["NumericValue"]))

            # Nếu dữ liệu rỗng
            else:
                # Điền vào giá trị None ở mỗi hàng ứng với từng quốc gia vào mảng nguyên nhân tử vong đó
                NUMBER_OF_DEATHS[cause_name].extend(list(None for _ in range(len(COUNTRIES["Country_ID"]))))

        # Điền các giá trị tên quốc gia từ COUNTRIES vào NUMBER_OF_DEATHS
        NUMBER_OF_DEATHS["CountryName"].extend(COUNTRIES["Country_Name"])

        # Điền các giá trị của năm vào NUMBER_OF_DEATHS cho từng quốc gia
        NUMBER_OF_DEATHS["Year"].extend(list(year for i in range(0, len(COUNTRIES["Country_Name"]))))

        print("Loading data in", year)

    # Trả về DataFrame tạo bởi dict NUMBER_OF_DEATHS
    return pd.DataFrame(NUMBER_OF_DEATHS)


In [8]:
#TEST
data_number_of_deaths = collect_data()

# 17 cột nguyên nhân tử vong + 1 cột tên quốc gia + 1 cột năm = 19 cột
# 194 quốc gia * 22 năm khảo sát = 4268 hàng
assert data_number_of_deaths.shape == (4268, 19)

Loading data in 2000
Loading data in 2001
Loading data in 2002
Loading data in 2003
Loading data in 2004
Loading data in 2005
Loading data in 2006
Loading data in 2007
Loading data in 2008
Loading data in 2009
Loading data in 2010
Loading data in 2011
Loading data in 2012
Loading data in 2013
Loading data in 2014
Loading data in 2015
Loading data in 2016
Loading data in 2017
Loading data in 2018
Loading data in 2019
Loading data in 2020
Loading data in 2021


In [9]:
# Lưu DataFrame vào file csv
data_number_of_deaths.to_csv("../../../Data/number_of_deaths_raw_data.csv", index=False)
data_number_of_deaths

,Prematurity,Birth asphyxia and birth trauma,Sepsis and other infectious conditions of the newborn,Other Group 1 (postneonatal only),Congenital anomalies,Other noncommunicable diseases (postneonatal only),Injuries,Tuberculosis,Other Group 1 and Other noncommunicable (neonatal and under-5 only),HIV/AIDS,Diarrhoeal diseases,Pertussis,Tetanus,Measles,Meningitis/encephalitis,Malaria,Acute lower respiratory infections,CountryName,Year
0,19333.41926,11691.37210,4489.50404,None,1534.08344,None,371.28902,0.0,10579.92644,0.22891,1973.08786,None,2331.69796,0.0,1395.33839,0.0,6908.05259,Afghanistan,2000
1,14761.72219,10042.87869,2977.92908,None,1405.50824,None,130.32201,0.0,3849.14708,30.31433,724.01316,None,768.57928,0.0,357.27433,0.0,4267.31161,Angola,2000
2,351.16760,110.05251,17.54192,None,106.22332,None,4.31283,0.0,30.48989,0.00347,0.77210,None,0.09764,0.0,10.36631,0.0,36.97242,Albania,2000
3,3.00000,0.00000,0.00000,None,0.00000,None,0.00000,0.0,0.00000,0.00000,0.00000,None,0.00000,0.0,0.00000,0.0,0.00000,Andorra,2000
4,137.90854,26.85080,2.86905,None,75.66284,None,1.78464,0.0,66.57905,0.10435,0.00000,None,0.00000,0.0,0.15043,0.0,0.09029,United Arab Emirates,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4263,17.45855,5.56010,2.93639,None,7.28102,None,0.25978,0.0,4.18806,0.00000,0.00000,None,0.00000,0.0,0.15554,0.0,1.16055,Samoa,2021
4264,11286.37393,6796.63328,1742.58956,None,2227.89226,None,229.08594,0.0,3051.97336,0.90444,521.35998,None,129.94279,0.0,44.07145,0.0,2523.17301,Yemen,2021
4265,4634.07215,2595.54661,1828.98610,None,1315.87236,None,166.21530,0.0,1708.85383,36.14888,0.00000,None,0.00000,0.0,16.01934,0.0,677.28544,South Africa,2021
4266,6920.62510,4054.12780,912.70245,None,1274.53105,None,35.09944,0.0,1776.66284,22.93804,134.08604,None,23.52546,0.0,108.91078,0.0,1228.79101,Zambia,2021
